In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("..")
import torch
import torch.utils.data
# from torchvision import datasets, transforms
import numpy as np
import argparse
from udlp.autoencoder.denoisingAutoencoder import DenoisingAutoencoder
from pacdataset import PacDataset

/home/paperspace/anaconda3/envs/fastai/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
batch_size = 4
lr = 0.001
epochs = 2

# DEA with PAC

In [3]:
# "/Users/josephmann/Documents/Gheiratmand/sMRI competition/PAC Data/pac2018/"
datasets = {x: PacDataset(train=(x=='train'), 
                          root_dir = "/home/paperspace/data/pac2018/") 
#                           root_dir = "/Users/josephmann/Documents/Gheiratmand/sMRI competition/PAC Data/pac2018/") 
            for x in ['val','train']}

train_loader = torch.utils.data.DataLoader(
    datasets['train'],
    batch_size= batch_size, shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(
    datasets['val'],
    batch_size= batch_size, shuffle=False, num_workers=2)

In [5]:
in_features = 17545
out_features = 500
dae = DenoisingAutoencoder(in_features, out_features)
dae.fit(train_loader, test_loader, lr=lr, num_epochs=epochs, loss_type="cross-entropy")

=====Denoising Autoencoding layer=======
####Epoch 0: Valid Reconstruct Loss: 12169.798
#Epoch   1: Reconstruct Loss: 3520.461, Valid Reconstruct Loss: 3417.626
#Epoch   2: Reconstruct Loss: 3334.102, Valid Reconstruct Loss: 3370.125


# SDEA with PAC

In [4]:
from udlp.autoencoder.stackedDAE import StackedDAE


In [12]:
# in_features = 784
in_features = 17545
out_features = 500
pretrainepochs = 20 
epochs = 20
z_dim = 100
batch_size = 32

sdae = StackedDAE(input_dim=in_features, z_dim=z_dim, binary=True,
    encodeLayer=[4000,1000,500], decodeLayer=[500,1000,4000], activation="relu", 
    dropout=0)
sdae.pretrain(train_loader, test_loader, lr=lr, batch_size= batch_size, 
    num_epochs=pretrainepochs, corrupt=0.3, loss_type="cross-entropy")
sdae.save_model("model/sdae.pt")
sdae.fit(train_loader, test_loader, lr= lr, num_epochs= epochs, corrupt=0.3, loss_type="cross-entropy")


l is  1
=====Denoising Autoencoding layer=======
####Epoch 0: Valid Reconstruct Loss: 12172.528
#Epoch   1: Reconstruct Loss: 3411.094, Valid Reconstruct Loss: 3393.106
#Epoch   2: Reconstruct Loss: 3278.276, Valid Reconstruct Loss: 3316.640
#Epoch   3: Reconstruct Loss: 3241.960, Valid Reconstruct Loss: 3299.227
#Epoch   4: Reconstruct Loss: 3206.112, Valid Reconstruct Loss: 3261.768
#Epoch   5: Reconstruct Loss: 3180.514, Valid Reconstruct Loss: 3263.220
#Epoch   6: Reconstruct Loss: 3156.363, Valid Reconstruct Loss: 3211.948
#Epoch   7: Reconstruct Loss: 3131.904, Valid Reconstruct Loss: 3188.156
#Epoch   8: Reconstruct Loss: 3112.562, Valid Reconstruct Loss: 3210.073
#Epoch   9: Reconstruct Loss: 3095.244, Valid Reconstruct Loss: 3185.712
#Epoch  10: Reconstruct Loss: 3080.548, Valid Reconstruct Loss: 3178.328
#Epoch  11: Reconstruct Loss: 3066.563, Valid Reconstruct Loss: 3154.954
#Epoch  12: Reconstruct Loss: 3052.418, Valid Reconstruct Loss: 3166.015
#Epoch  13: Reconstruct Loss

In [13]:
sdae.save_model("model/pre_vade.pt")

In [14]:
sdae.fit(train_loader, test_loader, lr= lr, num_epochs= epochs, corrupt=0.3, loss_type="cross-entropy")


=====Stacked Denoising Autoencoding layer=======
#Epoch 0: Valid Reconstruct Loss: 3171.951
#Epoch   1: Reconstruct Loss: 2987.999, Valid Reconstruct Loss: 3159.586
#Epoch   2: Reconstruct Loss: 2972.008, Valid Reconstruct Loss: 3149.414
#Epoch   3: Reconstruct Loss: 2972.544, Valid Reconstruct Loss: 3163.527
#Epoch   4: Reconstruct Loss: 2960.754, Valid Reconstruct Loss: 3176.698
#Epoch   5: Reconstruct Loss: 2961.723, Valid Reconstruct Loss: 3199.294
#Epoch   6: Reconstruct Loss: 2940.072, Valid Reconstruct Loss: 3162.047
#Epoch   7: Reconstruct Loss: 2934.605, Valid Reconstruct Loss: 3173.188
#Epoch   8: Reconstruct Loss: 2905.482, Valid Reconstruct Loss: 3163.828
#Epoch   9: Reconstruct Loss: 2898.027, Valid Reconstruct Loss: 3159.685
#Epoch  10: Reconstruct Loss: 2888.576, Valid Reconstruct Loss: 3177.659
#Epoch  11: Reconstruct Loss: 2870.851, Valid Reconstruct Loss: 3178.913
#Epoch  12: Reconstruct Loss: 2850.923, Valid Reconstruct Loss: 3175.868
#Epoch  13: Reconstruct Loss: 28

## Visualize a reconstruction...

In [37]:
# face_dataset = FaceLandmarksDataset(csv_file='faces/face_landmarks.csv',
#                                     root_dir='faces/')
from matplotlib import pyplot as plt
from torch.autograd import Variable
fig = plt.figure(figsize=(12,12))

use_cuda = torch.cuda.is_available()
for i in range(4): #len(datasets[0])):
    inputs = datasets['train'][i]
    inputs = inputs.view(inputs.size(0), -1).float()
    if use_cuda:
        inputs = inputs.cuda()
    inputs = Variable(inputs)
    z, outputs = sdae.forward(inputs)
    f_sample = sdae.forward(datasets['train'][i])
    sample = datasets['train'][i].numpy().reshape(121,145)

    print(i, sample.shape)

    ax = plt.subplot(1, 4, i + 1)
#     plt.tight_layout()
    ax.set_title('Sample #{}'.format(i))
    ax.axis('off')
    plt.imshow(sample)
#     show_landmarks(**sample)

    if i == 3:
        plt.show()
        break

RuntimeError: size mismatch at /opt/conda/conda-bld/pytorch_1522182087074/work/torch/lib/THC/generic/THCTensorMathBlas.cu:247

In [ ]:
sdae.

# Test VaDE with PAC and pre-trained model
just for file format



In [40]:
from udlp.clustering.vade import VaDE
args_pretrain = 'model/pre_vade.pt'

vade = VaDE(input_dim=in_features, z_dim=10, n_centroids=10, binary=True,
        encodeLayer=[500,500,2000], decodeLayer=[2000,500,500])

if args_pretrain != "":
    print("Loading model from %s..." % args_pretrain)
    vade.load_model(args_pretrain)
print("Initializing through GMM..")
vade.initialize_gmm(train_loader)
vade.fit(train_loader, test_loader, lr=lr, batch_size=batch_size, num_epochs=epochs, anneal=True)

Loading model from model/pre_vade.pt...
Initializing through GMM..
#Epoch -1: Valid Loss: 4140.25597
#Epoch   0: lr: 0.00100, Train Loss: 3241.66117, Valid Loss: 3192.90484, acc: 0.00000
#Epoch   1: lr: 0.00100, Train Loss: 3184.89950, Valid Loss: 3206.42785, acc: 0.00000
#Epoch   2: lr: 0.00100, Train Loss: 3169.68459, Valid Loss: 3202.02557, acc: 0.00000
#Epoch   3: lr: 0.00100, Train Loss: 3170.44605, Valid Loss: 3185.52840, acc: 0.00000
#Epoch   4: lr: 0.00100, Train Loss: 3162.35217, Valid Loss: 3196.19272, acc: 0.00000
#Epoch   5: lr: 0.00100, Train Loss: 3171.72202, Valid Loss: 3184.59456, acc: 0.00000
#Epoch   6: lr: 0.00100, Train Loss: 3168.05140, Valid Loss: 3174.83904, acc: 0.00000
#Epoch   7: lr: 0.00100, Train Loss: 3158.79186, Valid Loss: 3212.82876, acc: 0.00000
#Epoch   8: lr: 0.00100, Train Loss: 3151.82832, Valid Loss: 3219.81215, acc: 0.00000
#Epoch   9: lr: 0.00100, Train Loss: 3155.57900, Valid Loss: 3219.58857, acc: 0.00000
#Epoch  10: lr: 0.00090, Train Loss: 31

In [41]:
vade.save_model("model/vade2k5c5c_1.pt")

In [37]:
vade.load_model("model/vade1.pt")
print("Initializing through GMM..")
batch_size = 32
vade.initialize_gmm(train_loader)
vade.fit(train_loader, test_loader, lr=lr, batch_size=batch_size, num_epochs=epochs, anneal=True)

In [10]:
from udlp.clustering.vade import VaDE
args_pretrain = 'model/pre_vade.pt'

vade = VaDE(input_dim=in_features, z_dim=20, n_centroids=50, binary=True,
        encodeLayer=[2000,500,2000], decodeLayer=[2000,500,2000])

lr = 0.0005

if args_pretrain != "":
    print("Loading model from %s..." % args_pretrain)
    vade.load_model(args_pretrain)
print("Initializing through GMM..")
vade.initialize_gmm(train_loader)
vade.fit(train_loader, test_loader, lr=lr, batch_size=batch_size, num_epochs=epochs, anneal=True)

Loading model from model/pre_vade.pt...
Initializing through GMM..
#Epoch -1: Valid Loss: 4322.98422
#Epoch   0: lr: 0.00050, Train Loss: 3345.01400, Valid Loss: 3185.95488, acc: 0.00000
#Epoch   1: lr: 0.00050, Train Loss: 3186.08449, Valid Loss: 3180.28178, acc: 0.00000
#Epoch   2: lr: 0.00050, Train Loss: 3179.28265, Valid Loss: 3180.05393, acc: 0.00000
#Epoch   3: lr: 0.00050, Train Loss: nan, Valid Loss: nan, acc: 0.00000
#Epoch   4: lr: 0.00050, Train Loss: nan, Valid Loss: nan, acc: 0.00000


Process Process-509:
Process Process-510:
Traceback (most recent call last):
  File "/home/paperspace/anaconda3/envs/fastai/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/paperspace/anaconda3/envs/fastai/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/paperspace/anaconda3/envs/fastai/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 50, in _worker_loop
    r = index_queue.get()
  File "/home/paperspace/anaconda3/envs/fastai/lib/python3.6/multiprocessing/queues.py", line 335, in get
    res = self._reader.recv_bytes()
  File "/home/paperspace/anaconda3/envs/fastai/lib/python3.6/multiprocessing/connection.py", line 216, in recv_bytes
    buf = self._recv_bytes(maxlength)
  File "/home/paperspace/anaconda3/envs/fastai/lib/python3.6/multiprocessing/connection.py", line 407, in _recv_bytes
    buf = self._recv(4)
  File "/home/paperspace/anaconda3/envs/fast

KeyboardInterrupt: 

  File "/home/paperspace/anaconda3/envs/fastai/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 55, in _worker_loop
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "/home/paperspace/anaconda3/envs/fastai/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 55, in <listcomp>
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "/home/paperspace/github/UnsupervisedDeepLearning-Pytorch/test/pacdataset.py", line 39, in __getitem__
    img_data = PacDataset._jload(file_id)
  File "/home/paperspace/github/UnsupervisedDeepLearning-Pytorch/test/pacdataset.py", line 15, in _jload
    img_data = img.get_data()
  File "/home/paperspace/anaconda3/envs/fastai/lib/python3.6/site-packages/nibabel/dataobj_images.py", line 202, in get_data
    data = np.asanyarray(self._dataobj)
  File "/home/paperspace/anaconda3/envs/fastai/lib/python3.6/site-packages/numpy/core/numeric.py", line 544, in asanyarray
    return array(a, dtype, copy=False

In [51]:
from udlp.clustering.vade import VaDE
args_pretrain = 'model/pre_vade.pt'

z_dim = 20
n_centroids = 20
vade = VaDE(input_dim=in_features, z_dim=z_dim, n_centroids=n_centroids, binary=True,
        encodeLayer=[2000,500,2000], decodeLayer=[2000,500,2000])

# if args_pretrain != "":
#     print("Loading model from %s..." % args_pretrain)
#     vade.load_model(args_pretrain)
print("Initializing through GMM..")
vade.initialize_gmm(train_loader)
vade.fit(train_loader, test_loader, lr=lr, batch_size=batch_size, num_epochs=epochs, anneal=True)

Initializing through GMM..
#Epoch -1: Valid Loss: nan
#Epoch   0: lr: 0.00100, Train Loss: 6242.55512, Valid Loss: 3307.95430, acc: 0.00000
#Epoch   1: lr: 0.00100, Train Loss: 3317.17495, Valid Loss: 3339.55438, acc: 0.00000
#Epoch   2: lr: 0.00100, Train Loss: 3308.81975, Valid Loss: 3289.09791, acc: 0.00000
#Epoch   3: lr: 0.00100, Train Loss: 3303.23821, Valid Loss: 3262.72100, acc: 0.00000
#Epoch   4: lr: 0.00100, Train Loss: 3311.81923, Valid Loss: 3269.18336, acc: 0.00000
#Epoch   5: lr: 0.00100, Train Loss: 3293.82434, Valid Loss: 3321.69122, acc: 0.00000
#Epoch   6: lr: 0.00100, Train Loss: 3287.29481, Valid Loss: 3243.17547, acc: 0.00000
#Epoch   7: lr: 0.00100, Train Loss: 3272.09096, Valid Loss: 3257.97516, acc: 0.00000
#Epoch   8: lr: 0.00100, Train Loss: 3269.02134, Valid Loss: 3237.02857, acc: 0.00000
#Epoch   9: lr: 0.00100, Train Loss: 3261.04076, Valid Loss: 3233.40067, acc: 0.00000
#Epoch  10: lr: 0.00090, Train Loss: 3260.28980, Valid Loss: 3232.83958, acc: 0.00000


In [52]:
vade.save_model("model/vade2k5c2k_2020_1.pt")

In [54]:
vade.load_model("model/vade2k5c2k_2020_1.pt")
vade.initialize_gmm(train_loader)
vade.fit(train_loader, test_loader, lr=lr, batch_size=batch_size, num_epochs=epochs, anneal=True)

#Epoch -1: Valid Loss: nan
#Epoch   0: lr: 0.00100, Train Loss: 3311.62904, Valid Loss: 3223.40162, acc: 0.00000
#Epoch   1: lr: 0.00100, Train Loss: nan, Valid Loss: nan, acc: 0.00000


Process Process-1772:
  File "/home/paperspace/github/UnsupervisedDeepLearning-Pytorch/test/pacdataset.py", line 15, in _jload
    img_data = img.get_data()
Process Process-1771:
Traceback (most recent call last):
  File "/home/paperspace/anaconda3/envs/fastai/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/home/paperspace/anaconda3/envs/fastai/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/paperspace/anaconda3/envs/fastai/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/paperspace/anaconda3/envs/fastai/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 55, in _worker_loop
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "/home/paperspace/anaconda3/envs/fastai/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 55, in <listcomp>
    samples = c

KeyboardInterrupt: 

  File "/home/paperspace/anaconda3/envs/fastai/lib/python3.6/site-packages/nibabel/dataobj_images.py", line 202, in get_data
    data = np.asanyarray(self._dataobj)
  File "/home/paperspace/anaconda3/envs/fastai/lib/python3.6/site-packages/nibabel/volumeutils.py", line 965, in apply_read_scaling
    arr = arr * slope
  File "/home/paperspace/anaconda3/envs/fastai/lib/python3.6/site-packages/numpy/core/numeric.py", line 544, in asanyarray
    return array(a, dtype, copy=False, order=order, subok=True)
  File "/home/paperspace/anaconda3/envs/fastai/lib/python3.6/site-packages/numpy/core/memmap.py", line 319, in __array_wrap__
    def __array_wrap__(self, arr, context=None):
  File "/home/paperspace/anaconda3/envs/fastai/lib/python3.6/site-packages/nibabel/arrayproxy.py", line 291, in __array__
    return apply_read_scaling(raw_data, self._slope, self._inter)
KeyboardInterrupt
  File "/home/paperspace/anaconda3/envs/fastai/lib/python3.6/site-packages/nibabel/volumeutils.py", line 965, in 

In [44]:
vade.save_model("model/vade2k5c2k_1.pt")

In [45]:
vade.load_model("model/vade2k5c2k_1.pt")
print("Initializing through GMM..")
lr = 0.001
batch_size = 32
vade.initialize_gmm(train_loader)
vade.fit(train_loader, test_loader, lr=lr, batch_size=batch_size, num_epochs=epochs, anneal=True)

Initializing through GMM..
#Epoch -1: Valid Loss: 6526.86497
#Epoch   0: lr: 0.00100, Train Loss: 3262.76987, Valid Loss: 3232.34285, acc: 0.00000
#Epoch   1: lr: 0.00100, Train Loss: 3242.06520, Valid Loss: 3211.50148, acc: 0.00000
#Epoch   2: lr: 0.00100, Train Loss: 3242.81475, Valid Loss: 3210.97557, acc: 0.00000
#Epoch   3: lr: 0.00100, Train Loss: 3236.35666, Valid Loss: 3204.28867, acc: 0.00000
#Epoch   4: lr: 0.00100, Train Loss: 3235.02464, Valid Loss: 3200.39091, acc: 0.00000
#Epoch   5: lr: 0.00100, Train Loss: 3247.62324, Valid Loss: 3212.14897, acc: 0.00000
#Epoch   6: lr: 0.00100, Train Loss: 3234.69476, Valid Loss: 3191.59323, acc: 0.00000
#Epoch   7: lr: 0.00100, Train Loss: 3224.14547, Valid Loss: 3188.05998, acc: 0.00000
#Epoch   8: lr: 0.00100, Train Loss: 3223.49146, Valid Loss: 3187.75288, acc: 0.00000
#Epoch   9: lr: 0.00100, Train Loss: 3220.72840, Valid Loss: 3191.08219, acc: 0.00000
#Epoch  10: lr: 0.00090, Train Loss: 3217.33778, Valid Loss: 3184.41324, acc: 0

In [46]:
vade.save_model("model/vade2k5c2k_2.pt")

In [48]:
vade.load_model("model/vade2k5c2k_2.pt")
print("Initializing through GMM..")
lr = 0.001
batch_size = 32
vade.initialize_gmm(train_loader)
vade.fit(train_loader, test_loader, lr=lr, batch_size=batch_size, num_epochs=epochs, anneal=True)

Initializing through GMM..
#Epoch -1: Valid Loss: 8679.95242
#Epoch   0: lr: 0.00100, Train Loss: 3233.41579, Valid Loss: 3186.49018, acc: 0.00000
#Epoch   1: lr: 0.00100, Train Loss: 3209.28563, Valid Loss: 3183.59263, acc: 0.00000
#Epoch   2: lr: 0.00100, Train Loss: 3211.07545, Valid Loss: 3183.09875, acc: 0.00000
#Epoch   3: lr: 0.00100, Train Loss: 3211.02946, Valid Loss: 3182.68724, acc: 0.00000
#Epoch   4: lr: 0.00100, Train Loss: 3211.01418, Valid Loss: 3182.51615, acc: 0.00000
#Epoch   5: lr: 0.00100, Train Loss: 3209.16660, Valid Loss: 3183.76033, acc: 0.00000
#Epoch   6: lr: 0.00100, Train Loss: 3208.18500, Valid Loss: 3181.53952, acc: 0.00000
#Epoch   7: lr: 0.00100, Train Loss: 3210.69156, Valid Loss: 3182.22657, acc: 0.00000
#Epoch   8: lr: 0.00100, Train Loss: 3216.56231, Valid Loss: 3192.80368, acc: 0.00000
#Epoch   9: lr: 0.00100, Train Loss: 3228.06418, Valid Loss: 3204.26479, acc: 0.00000
#Epoch  10: lr: 0.00090, Train Loss: 3213.43192, Valid Loss: 3181.38049, acc: 0

In [49]:
vade.save_model("model/vade2k5c2k_3.pt")

In [50]:
vade.load_model("model/vade2k5c2k_3.pt")
print("Initializing through GMM..")
lr = 0.001
batch_size = 32
vade.initialize_gmm(train_loader)
vade.fit(train_loader, test_loader, lr=lr, batch_size=batch_size, num_epochs=epochs, anneal=True)

Initializing through GMM..
#Epoch -1: Valid Loss: 13186.21874
#Epoch   0: lr: 0.00100, Train Loss: 3230.66995, Valid Loss: 3191.42460, acc: 0.00000
#Epoch   1: lr: 0.00100, Train Loss: 3198.93020, Valid Loss: 3182.39906, acc: 0.00000
#Epoch   2: lr: 0.00100, Train Loss: 3200.67083, Valid Loss: 3181.66593, acc: 0.00000
#Epoch   3: lr: 0.00100, Train Loss: 3199.40258, Valid Loss: 3183.60423, acc: 0.00000
#Epoch   4: lr: 0.00100, Train Loss: 3195.83137, Valid Loss: 3179.73581, acc: 0.00000
#Epoch   5: lr: 0.00100, Train Loss: 3197.30092, Valid Loss: 3181.60865, acc: 0.00000
#Epoch   6: lr: 0.00100, Train Loss: 3194.53059, Valid Loss: 3180.47269, acc: 0.00000
#Epoch   7: lr: 0.00100, Train Loss: 3202.62261, Valid Loss: 3182.16113, acc: 0.00000
#Epoch   8: lr: 0.00100, Train Loss: 3194.71276, Valid Loss: 3183.98300, acc: 0.00000
#Epoch   9: lr: 0.00100, Train Loss: 3197.05575, Valid Loss: 3185.75998, acc: 0.00000
#Epoch  10: lr: 0.00090, Train Loss: 3194.80184, Valid Loss: 3179.34724, acc: 

> I don't know the number of centroids does...

In [47]:
vade.load_model("model/vade2k5c2k_1.pt")
print("Initializing through GMM..")
lr = 0.001
batch_size = 32
# we do NOT want to initialize_gmm each time, oh wait, yes we do.
# vade.initialize_gmm(train_loader)
vade.fit(train_loader, test_loader, lr=lr, batch_size=batch_size, num_epochs=epochs, anneal=True)

Initializing through GMM..
#Epoch -1: Valid Loss: 3219.21289
#Epoch   0: lr: 0.00100, Train Loss: nan, Valid Loss: nan, acc: 0.00000
#Epoch   1: lr: 0.00100, Train Loss: nan, Valid Loss: nan, acc: 0.00000
#Epoch   2: lr: 0.00100, Train Loss: nan, Valid Loss: nan, acc: 0.00000
#Epoch   3: lr: 0.00100, Train Loss: nan, Valid Loss: nan, acc: 0.00000
#Epoch   4: lr: 0.00100, Train Loss: nan, Valid Loss: nan, acc: 0.00000


Process Process-1496:
Process Process-1495:
Traceback (most recent call last):
  File "/home/paperspace/anaconda3/envs/fastai/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/paperspace/anaconda3/envs/fastai/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/paperspace/anaconda3/envs/fastai/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 50, in _worker_loop
    r = index_queue.get()
  File "/home/paperspace/anaconda3/envs/fastai/lib/python3.6/multiprocessing/queues.py", line 335, in get
    res = self._reader.recv_bytes()
  File "/home/paperspace/anaconda3/envs/fastai/lib/python3.6/multiprocessing/connection.py", line 216, in recv_bytes
    buf = self._recv_bytes(maxlength)
  File "/home/paperspace/anaconda3/envs/fastai/lib/python3.6/multiprocessing/connection.py", line 407, in _recv_bytes
    buf = self._recv(4)
  File "/home/paperspace/anaconda3/envs/fa

KeyboardInterrupt: 

In [32]:
# batch_size = 8
# vade.load_model("model/vade1.pt")
# print("Initializing through GMM..")
# vade.initialize_gmm(train_loader)
vade.fit(train_loader, test_loader, lr=lr, batch_size=batch_size, num_epochs=epochs, anneal=True)

#Epoch -1: Valid Loss: 3208.37063
#Epoch   0: lr: 0.00100, Train Loss: 3107.86518, Valid Loss: 3172.76114, acc: 0.00000
#Epoch   1: lr: 0.00100, Train Loss: 3110.91332, Valid Loss: 3176.45886, acc: 0.00000
#Epoch   2: lr: 0.00100, Train Loss: 3110.19600, Valid Loss: 3178.79659, acc: 0.00000
#Epoch   3: lr: 0.00100, Train Loss: 3104.94653, Valid Loss: 3183.62467, acc: 0.00000
#Epoch   4: lr: 0.00100, Train Loss: 3118.62083, Valid Loss: 3178.66328, acc: 0.00000
#Epoch   5: lr: 0.00100, Train Loss: 3114.10822, Valid Loss: 3199.02968, acc: 0.00000
#Epoch   6: lr: 0.00100, Train Loss: 3120.37661, Valid Loss: 3173.95579, acc: 0.00000
#Epoch   7: lr: 0.00100, Train Loss: 3107.35450, Valid Loss: 3185.31540, acc: 0.00000
#Epoch   8: lr: 0.00100, Train Loss: 3095.57662, Valid Loss: 3168.91423, acc: 0.00000
#Epoch   9: lr: 0.00100, Train Loss: 3088.22531, Valid Loss: 3190.91982, acc: 0.00000
#Epoch  10: lr: 0.00090, Train Loss: 3084.56503, Valid Loss: 3168.53700, acc: 0.00000
#Epoch  11: lr: 0.00